In [1]:
import pandas as pd
train=pd.read_csv('train.csv')
from sklearn import preprocessing
#transform characters into numbers
le=preprocessing.LabelEncoder()
for i in range(train.shape[1]):
    if train.iloc[:,i].dtype=='O':
        train.iloc[:,i]=le.fit_transform(train.iloc[:,i])

test=pd.read_csv('test.csv')
for i in range(test.shape[1]):
    if test.iloc[:,i].dtype=='O':
        test.iloc[:,i]=le.fit_transform(test.iloc[:,i])

In [2]:
#split the train set into train and validation
from sklearn.cross_validation import train_test_split
train_train,train_val=train_test_split(train,test_size=0.33)
predictors=train.columns.values.tolist()[2:]

In [3]:
from sklearn.ensemble import RandomForestRegressor
alg=RandomForestRegressor(n_estimators=100, max_features = 'sqrt', oob_score=True, verbose = 1)
alg.fit(train_train[predictors],train_train['Hazard'])

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    5.6s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   11.2s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=True, random_state=None,
           verbose=1, warm_start=False)

In [4]:
#predict on validation_set
predictions=alg.predict(train_val[predictors])

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished


In [5]:
# Simple implementation of the (normalized) gini score in numpy
# Fully vectorized, no python loops, zips, etc.
# Significantly (>30x) faster than previous implementions

import numpy as np 

def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    
    # sort true values on prediction column 
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:,0].argsort()][::-1,0]
    pred_order = arr[arr[:,1].argsort()][::-1,0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) / np.sum(true_order)
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    L_ones = np.linspace(1/n_samples, 1, n_samples)
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)
    
    # normalize to true Gini coefficient
    return G_pred/G_true

In [6]:
#What do we have?
Gini(train_val['Hazard'],predictions)

0.3197467298055684

In [7]:
#Now let's play with our forests
alg=RandomForestRegressor(n_estimators=100, verbose = 1)
alg.fit(train_train[predictors],train_train['Hazard'])

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   26.9s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   53.8s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=1, warm_start=False)

In [8]:
predictions=alg.predict(train_val[predictors])
print(Gini(train_val['Hazard'],predictions))

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.4s


0.314703469702


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished


In [9]:
from sklearn.ensemble import RandomForestRegressor
alg=RandomForestRegressor(n_estimators=10, oob_score=True, verbose = 1)
alg.fit(train_train[predictors],train_train['Hazard'])
predictions=alg.predict(train_val[predictors]).round()
print(Gini(train_val['Hazard'],predictions))

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.3s finished
/home/egor/anaconda3/lib/python3.4/site-packages/sklearn/ensemble/forest.py:659: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


0.226218304942


In [10]:
#Phew, cool
#Now let's try to train on the whole train data and make a prediction on test
alg.fit(train[predictors],train['Hazard'])

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:  1.2min
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.4min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=1, warm_start=False)

In [11]:
predictions=alg.predict(test[predictors])

submission = pd.DataFrame({
        "Id": test["Id"],
        "Hazard": predictions
    })

submission.to_csv("submission_python_rf.csv", index=False, columns=['Id','Hazard'])

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    1.7s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.5s finished


0.339149, less than I had with random forest from R.

What if we try to fit classifier instead of regressor? After all, there's a lot of ones in the data and not much of other hazard levels

In [10]:
from sklearn.ensemble import RandomForestClassifier
alg=RandomForestClassifier(n_estimators=100, max_features = 'sqrt', oob_score=True, verbose = 1)
alg.fit(train_train[predictors],train_train['Hazard'])

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   11.3s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   35.4s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=None, verbose=1, warm_start=False)

In [11]:
predictions=alg.predict(train_val[predictors])
print(predictions[:10])
print(train_val['Hazard'][:10])
print(Gini(train_val['Hazard'],predictions))

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    4.4s


[1 1 1 1 1 1 1 1 1 4]
35096    1
38644    1
39823    1
50014    4
47841    1
30753    1
6301     8
36897    1
33004    4
19042    2
Name: Hazard, dtype: int64
0.237323963156


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   13.6s finished


In [21]:
#stupid try with just ten trees and rounding of results
alg=RandomForestRegressor(n_estimators=150, oob_score=True, verbose = 1)
alg.fit(train[predictors],train['Hazard'])
predictions=alg.predict(test[predictors]).round()

submission = pd.DataFrame({
        "Id": test["Id"],
        "Hazard": predictions
    })

submission.to_csv("submission_python_rf.csv", index=False, columns=['Id','Hazard'])

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    1.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   51.8s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    1.5s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    4.5s finished


In [24]:
import gc
gc.enable()
gc.collect()

9